**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

---


# Introduction

You are getting to the point where you can own an analysis from beginning to end. So you'll do more data exploration in this exercise than you've done before.  Before you get started, run the following set-up code as usual. Notice that your `BigQueryHelper` object is called `chicago_taxi_helper`.

In [1]:
# Set up feedack system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *

# import package with helper functions 
import bq_helper

# create a helper object for this dataset
chicago_taxi_helper = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                               dataset_name="chicago_taxi_trips")

Using Kaggle's public dataset BigQuery integration.
Using Kaggle's public dataset BigQuery integration.


# Questions

This dataset contains a table with the information about taxi trips in the city of Chicago. You are curious how much slower traffic moves when traffic volume is high. This involves a few steps.

# 1) Find the Data
Before you can access the data, you need to find the table name with the data.

- Hint: Tab completion is helpful whenever you can't remember a command. Type `chicago_taxi_helper.` and then hit the tab key. Don't forget the period before hitting tab.

In [2]:
# Your code here to find the table name
____
chicago_taxi_helper.list_tables()

['taxi_trips']

In [3]:
# write the table name as a string below
table_name = 'taxi_trips'

q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
chicago_taxi_helper.table_schema('taxi_trips')

,name,type,mode,description
0,unique_key,STRING,REQUIRED,Unique identifier for the trip.
1,taxi_id,STRING,REQUIRED,A unique identifier for the taxi.
2,trip_start_timestamp,TIMESTAMP,NULLABLE,"When the trip started, rounded to the nearest ..."
3,trip_end_timestamp,TIMESTAMP,NULLABLE,"When the trip ended, rounded to the nearest 15..."
4,trip_seconds,INTEGER,NULLABLE,Time of the trip in seconds.
5,trip_miles,FLOAT,NULLABLE,Distance of the trip in miles.
6,pickup_census_tract,INTEGER,NULLABLE,The Census Tract where the trip began. For pri...
7,dropoff_census_tract,INTEGER,NULLABLE,The Census Tract where the trip ended. For pri...
8,pickup_community_area,INTEGER,NULLABLE,The Community Area where the trip began.
9,dropoff_community_area,INTEGER,NULLABLE,The Community Area where the trip ended.


In [5]:
# q_1.solution()

# 2) Peek at the data

Use `chicago_taxi_helper.head` to peak at the top few rows of the data. Inspect the data and see if the data quality is immediately obvious. After deciding whether you see any important issues, look at `q_2.solution()`.

In [6]:
# your code here
chicago_taxi_helper.head('taxi_trips')

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,5617797925fa2ea420e31bde7804daf27fcb8536,73c788fff8c6a38a113b2a002b98894b888233f223d034...,2015-04-17 15:00:00+00:00,2015-04-17 15:00:00+00:00,0,0.0,None,None,None,None,845.0,211.0,0.0,0.0,1056.0,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
1,5618c6ea5e53573cc6c391eedfd30626f28120a6,803c154120d7dbec20a6e92780e4fd276ae330a9cd3185...,2014-07-27 03:15:00+00:00,2014-07-27 03:15:00+00:00,0,0.0,None,None,None,None,1985.0,496.0,0.0,0.0,2481.0,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
2,561eb8269316eeafa8ae5c1d82aedfd6eb6b9d14,19adbf30524c6579e4386e81001f72b9973a747ac9753a...,2015-10-06 19:30:00+00:00,2015-10-06 19:30:00+00:00,0,0.0,None,None,None,None,1265.0,0.0,0.0,0.0,1265.0,Cash,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
3,56208d54fa681a6cae78a0974a0a5f266b470243,f4bc2acfbb68d505b7ca37f8553ab1fc28a2686429b55d...,2015-12-07 13:00:00+00:00,2015-12-07 13:00:00+00:00,0,0.0,None,None,None,None,3985.0,0.0,0.0,0.0,3985.0,Cash,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
4,562d6dcd0c9d867fbfbf672212b9b5138d282962,9bc3b89d49a30e5990cd30232c8caf69a5774903e9de81...,2016-01-13 16:45:00+00:00,2016-01-13 16:45:00+00:00,0,0.0,None,None,None,None,6775.0,1355.0,0.0,0.0,8130.0,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None


In [7]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
You can see the data by calling: 
```python
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=5).to_dataframe()
```

Some trips in the top few rows have `trip_seconds` or `trip_miles` values of 0. 
Other location fields have values of `None`. That is a problem if we want to use those fields.


# 3) Determine when this data is from

If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.  

Your results should have two columns
1. The year of the trips, which is called `year`
2. The number of trips in that year, called `num_trips`.

Hints:
- When using `GROUP BY` and `ORDER BY`, you should refer to the columns by the alias `year` that you set at the top of the **SELECT** query.
- The sql code to select the Year from `trip_start_timestamp` is <code>SELECT **EXTRACT(YEAR FROM trip_start_timestamp)**</code>.
- The FROM field can be a little tricky until you are used to it.  The format is
    1. A backick (the symbol \`)
    2. The project name. You can find this in the line where we created the BigQueryHelper object. In this case it is `bigquery-public-data`
    3. A period
    4. The dataset name. Also in the line where we created the BigQueryHelper object. In this case, it is `chicago_taxi_trips`
    5. A period
    6. The table name, which you already used as your answer in part 1.



In [8]:
rides_per_year_query = """
SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, COUNT(1) AS num_trips
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
GROUP BY year
ORDER BY year
"""

rides_per_year_result = chicago_taxi_helper.query_to_pandas_safe(rides_per_year_query)


print(rides_per_year_result)
#q_3.check()

   year  num_trips
0  2013   26870287
1  2014   31021726
2  2015   27400744
3  2016   19878276
4  2017    7689021


In [9]:
q_3.hint()
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Start your query with `SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, COUNT(1) AS num_trips`.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

rides_per_year_query = """
                       SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, 
                              COUNT(1) AS num_trips
                       FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                       GROUP BY year
                       ORDER BY year
                       """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=1e9)
rides_per_year_query_job = client.query(rides_per_year_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
rides_per_year_result = rides_per_year_query_job.to_dataframe()

```

# 4) Dive Slightly Deeper

It's odd that 2017 had so few rides. You should wonder whether it was systematic under-reporting throughout the year, or whether some months are missing.  Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`.  Then modify it in two ways
1. Use a where clause to limit our query to data from 2017
2. Modify the extract statement to extract the month rather than the year


In [10]:
rides_per_month_query = """
SELECT EXTRACT(MONTH FROM trip_start_timestamp) AS month, COUNT(1) num_trips
from `bigquery-public-data.chicago_taxi_trips.taxi_trips`
where EXTRACT(YEAR FROM trip_start_timestamp) = 2017
GROUP BY month
ORDER BY month

"""

rides_per_month_result = chicago_taxi_helper.query_to_pandas_safe(rides_per_month_query)

print(rides_per_month_result)
#q_4.check()

   month  num_trips
0      1    1040262
1      2    1016484
2      3    1293745
3      4    1187501
4      5    1132690
5      6    1092557
6      7     925711
7      8         71


In [11]:
q_4.hint()
q_4.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Start your query with `SELECT EXTRACT(MONTH FROM trip_start_timestamp) AS month, COUNT(1) AS num_trips`.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

rides_per_month_query = """
                        SELECT EXTRACT(MONTH FROM trip_start_timestamp) AS month, 
                               COUNT(1) AS num_trips
                        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2017
                        GROUP BY month
                        ORDER BY month
                        """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=1e9)
rides_per_month_query_job = client.query(rides_per_month_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe()

```

# 5) Write the Query

Let's step up the sophistication of your queries.

Write a query
- A column called `hour_of_day` and sort by this, which holds the result of extracting the hour from `trip_start_timestamp`.
- The count of total number of trips in each hour of the day (e.g. how many trips occurred between 6AM and 7AM, independent of which day it occurred on). Call this `num_trips`
- The average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. Call this column `avg_mph`. The value 3600 is used to convert from seconds to hours.

For 2017, we're missing August and everything after. So restrict your query to data meeting the following criteria
- a `trip_start_timestamp` between **2017-01-01** and **2017-07-01**
- `trip_seconds` > 0 and `trip_miles` > 0


You will use a CTE to select just the relevant rides.  Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE. Instead you'll create those in the later `SELECT` statement).

This is a much harder query than anything you've written so far.  Good luck!


In [12]:
speeds_query = """WITH RelevantRides AS
(SELECT EXTRACT(HOUR FROM trip_start_timestamp) hour_of_day, trip_miles, trip_seconds
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE trip_start_timestamp > '2017-01-01' and trip_start_timestamp < '2017-07-01'
    AND trip_seconds > 0 AND trip_miles > 0
)

SELECT hour_of_day, count(1) num_trips, 3600 * SUM(trip_miles) / SUM(trip_seconds) avg_mph
FROM RelevantRides
GROUP BY hour_of_day
ORDER BY hour_of_day
"""

# Set high max_gb_scanned because this query looks at more data
speeds_result = chicago_taxi_helper.query_to_pandas_safe(speeds_query, max_gb_scanned=20)

print(speeds_result)
#q_5.check()

    hour_of_day  num_trips    avg_mph
0             0     155426  18.997085
1             1     124335  18.018430
2             2      96917  17.359694
3             3      72129  17.192603
4             4      50173  21.239755
5             5      42156  27.122256
6             6      63782  23.627121
7             7     138718  16.994303
8             8     236213  14.145707
9             9     271779  14.877002
10           10     253637  17.161201
11           11     271200  18.418800
12           12     297453  17.463159
13           13     303048  17.040498
14           14     301629  16.487089
15           15     306881  15.216843
16           16     338714  13.913319
17           17     378886  12.131222
18           18     397864  12.435833
19           19     371450  15.151219
20           20     320302  18.075683
21           21     282070  19.417275
22           22     253412  18.728543
23           23     204709  18.413801


In [13]:
q_5.hint()
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#cc5533">Sorry, no hints available for this question.</span>

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, 
                          trip_miles, 
                          trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01' AND 
                         trip_start_timestamp < '2017-07-01' AND 
                         trip_seconds > 0 AND 
                         trip_miles > 0
               )
               SELECT hour_of_day, 
                      COUNT(1) AS num_trips, 
                      3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=1e9)
speeds_query_job = client.query(speeds_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe()

# View results
print(speeds_result)

```

That's a hard query. If you made good progress towards the solution, congratulations!

# 6. Ponder the Results
Something is wrong with either the raw data or our last query. What fact about the raw data doesn't seem right?

If you can identify the problem, how would you look at the raw data to verify that the problem is in the raw data and not just in your results? Check your answer below.

In [14]:
# q_6.solution()

# Keep Going

You can write very complex queries now with a single data source. But nothing expands the horizons of SQL as much as the ability to combine or **JOIN** tables.

**[Click here](https://www.kaggle.com/dansbecker/joining-data)** to start the last lesson in SQL course.


---
**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

